Here we will estimate MPG of cars

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from IPython.display import Image

In [2]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch13/figures/13_07.png', width=500)

In [3]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

df = pd.read_csv(url, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)
df.head(10)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
5,15.0,8,429.0,198.0,4341.0,10.0,70,1
6,14.0,8,454.0,220.0,4354.0,9.0,70,1
7,14.0,8,440.0,215.0,4312.0,8.5,70,1
8,14.0,8,455.0,225.0,4425.0,10.0,70,1
9,15.0,8,390.0,190.0,3850.0,8.5,70,1


In [4]:
## Remove NA valued data

In [5]:
print(df.isna().sum())

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


In [6]:
df = df.dropna()
df = df.reset_index(drop=True)

### Separater traning and test data set

In [7]:
import sklearn
import sklearn.model_selection
df_train, df_test = sklearn.model_selection.train_test_split(
    df, train_size=0.8, random_state=1
)

In [8]:
print(len(df_train), len(df_test))

313 79


In [9]:
train_stats = df_train.describe().transpose()

In [10]:
numeric_column_names = [
    'Cylinders', 'Displacement',
    'Horsepower', 'Weight',
    'Acceleration'
]

In [11]:
## Normalize numerical value data

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] =\
        (df_train_norm.loc[:, col_name]-mean)/std
    df_test_norm.loc[:, col_name] =\
        (df_test_norm.loc[:, col_name]-mean)/std

In [12]:
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


In [13]:
## Bucketize Model year data

boundaries = torch.tensor([73, 76, 79])
# tran data set
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(
    v, boundaries, right=True
)

#test data set
v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(
    v, boundaries, right=True
)

In [14]:
numeric_column_names.append('Model Year Bucketed')

In [15]:
df_train_norm.tail(10)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin,Model Year Bucketed
71,15.0,1.526556,1.115038,1.241336,1.096865,-1.175781,72,1,0
129,32.0,-0.824303,-1.154244,-1.000281,-1.325943,1.943028,74,3,1
144,28.0,-0.824303,-0.969196,-0.736562,-0.985383,-0.441944,74,1,1
335,26.6,-0.824303,-0.375092,-0.499214,-0.384395,0.255202,81,1,3
133,18.0,0.351127,0.345625,0.054598,0.768089,0.291894,74,1,1
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3,2
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1,2
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1,0
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1,2
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1,0


In [16]:
## One Hot encoding Categorize Origin data

from torch.nn.functional import one_hot

total_origin = len(set(df_train_norm['Origin']))

origin_encoded = one_hot(torch.from_numpy(
    df_train_norm['Origin'].values) % total_origin)
                         
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
                         
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

In [17]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

### Making Deep Neural Network model for regression

In [18]:
from torch.utils.data import TensorDataset, DataLoader

train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [19]:
hidden_units = [8, 4]
input_size = x_train.shape[1]
all_layers = []

for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

In [20]:
all_layers.append(nn.Linear(hidden_units[-1],1))
model = nn.Sequential(*all_layers)

In [21]:
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [22]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [23]:
torch.manual_seed(1)
num_epochs = 200
log_epochs = 20
for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()
    if epoch % log_epochs ==0:
        print(f'Epoch {epoch} loss' f'{loss_hist_train/len(train_dl):.4f}')

Epoch 0 loss536.1047
Epoch 20 loss8.4361
Epoch 40 loss7.8695
Epoch 60 loss7.1891
Epoch 80 loss6.7062
Epoch 100 loss6.7599
Epoch 120 loss6.3124
Epoch 140 loss6.6864
Epoch 160 loss6.7648
Epoch 180 loss6.2156


In [24]:
with torch.no_grad():
    pred = model(x_test.float())[:, 0]
    loss = loss_fn(pred, y_test)
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test).item():4f}')

Test MSE: 9.6130
Test MAE: 2.121109
